本笔记记录了源自Udactiy自动驾驶课程中关于路径规划的入门级算法。主要针对“离散”的环境下的路径规划问题。涉及了A*，以及动态规划Dyanamic Programming两个典型算法。

# A*

### 离散环境的说明：
构建一个5*6的矩阵grid，代表整个“地图”,其中“0”表示可以移动到的位置，“1”则表示障碍物。该离散环境如下
```
[[0, 1, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0]]
```
其他变量的定义如下：
* init 为起始位置
* goal 为目标位置
* cost 表示每一动一步需要的代价
* delta表 示移动的方向，[-1,0]代表上，[0，-1]代表左移，[1，0]代表下移，[0,1]代表右移
* delta_name 用符号标出具体的移动方向

### A*算法说明
A*算法相比于一般的搜索算法，通过启发函数使得搜索的方向更接近于指向终点的方向，从而能更好地找到最短的路径。其关键在于需建立一个启发函数 heuristic function： 
* 启发函数的尺寸与地图相当。
* 每个单元格的数字代表了离终点的距离，终点本身标记为0，移动一个能到达终点则为1，移动2格能到达终点则为2，以此类推。 

为了得到最短的路径，要求在计算每一个步骤 action 的g值时（可以简单理解为从起点走到当前位置时已经经过的步数），需要多增加一个f值，既对应的启发函数（矩阵）中对应的数值

A*算法的大致步骤为：
1. 对各个变量进行初始化，其中将g值初始化为0
1. 判断开放节点列表len(open)是否为空，如为空，且已经到终点found=True，则结束循环e。如为空，且未到达终点,则无可行路径，返回"fail"
2. 如开放节点列表不为空，则里取f值最小的位置 [f,g,h,x,y]. 
3. 根据delta找到相邻的可以移动到的位置[x2,y2]，并计算移动到该位置的g2值
4. 通过启发函数（矩阵）得到新位置[x2,y2]的h2值
5. 将g2与h2值相加得到f2值
6. 将[f2,g2,h2,x2,y2]加入开放节点列表 open_list中
7. 将[x2,y2]标记为关闭节点
8. 进行下一次循环


In [1]:
# 定义地图，0为可移动的位置，1为障碍物
grid = [[0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0]]

#定义在离散地图中的移动方向
delta = [[-1, 0 ], # go up
         [ 0, -1], # go left
         [ 1, 0 ], # go down
         [ 0, 1 ]] # go right
delta_name = ['^', '<', 'v', '>']

# 根据目标位置定义启发函数，目标位置值为0，距离越远值越大
goal = [len(grid)-1, len(grid[0])-1]
heuristic = [[9, 8, 7, 6, 5, 4],
             [8, 7, 6, 5, 4, 3],
             [7, 6, 5, 4, 3, 2],
             [6, 5, 4, 3, 2, 1],
             [5, 4, 3, 2, 1, 0]]

# 设置初始位置
init = [0, 0]

#设置移动消耗
cost = 1

In [2]:
def a_star_search(grid,init,goal,cost,heuristic):

    # 构造expand矩阵，用于记录从起点起步，到达每个单元格的步数。初始化为-1，表示该点不可达。
    expand = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]

    # 构造close矩阵，表示已经走过过的节点，与地图grid相同，所有位置值初始化为0，起始位置初始值为1
    closed = [[0 for col in range(len(grid[0]))] for row in range(len(grid))]
    closed[init[0]][init[1]] = 1

    #初始化变量
    #初始化起点坐标
    x = init[0]
    y = init[1]
    #初始化起点g值
    g = 0
    #初始化h值，并计算f值
    h = heuristic[x][y]
    f = g + h
    #初始化开放节点，加入起始点init
    open = [[f,g,h,x,y]]

    #构建循环判断变量
    #found初始设置为false，表示还未找到终点
    found = False
    #resign初始设置为false，表示探索尚未中止（开放节点不为空）
    resign = False
    
    #初始化步数count为0
    count = 0

    #开始循环
    while found is False and resign is False: #判断是否为终点，以及是否中止

        #判断是否还有open的节点，如果没有，则将resign设为真，回到while循环开头，判断resign为真后退出循环
        if len(open) == 0: 
            resign = True
            print("fail") 

        #open的节点不为空，则继续探索open的节点
        else:
            #对open的节点进行排序，默认为从小到大，排序后需进行reverse改为从大到小排，通过pop取出最后一个元素，则为f最小的节点
            open.sort()
            open.reverse()
            next = open.pop()

            #取处开放节点的坐标x,y和当前的g值
            x = next[3]
            y = next[4]
            g = next[1]
            
            #在expand中把该位置改写为当前经过的步骤        
            expand[x][y] = count
            count += 1 

            #判断如果当前位置是目标位置，则把found定义为真，回到while循环初始位置后跳出while循环
            if x==goal[0] and y==goal[1]:
                found =True
            
            #如果当前位置不是目标位置，则进入主要的更新步骤
            else:
                #依次探索四个移动方向
                for i in range(len(delta)):
                    x2 = x + delta[i][0]
                    y2 = y + delta[i][1]
                    #首先判断个方向是否在地图范围内
                    if x2 >= 0 and x2 <= len(grid)-1 and y2 >=0 and y2 <= len(grid[0])-1:
                        #再判断下一个位置是不是还没被探索，且没有障碍物
                        if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                            #如果下一个位置是可达的，则将g值+1,并计算新的h和f值
                            g2 = g + cost
                            h2 = heuristic[x2][y2]
                            f2 = g2 + h2
                            #将这个相邻的节点以及其f值加入开放列表
                            open.append([f2, g2, h2, x2, y2])
                            #并将它标记为已探索
                            closed[x2][y2] = 1

    return expand

In [3]:
a_star_search(grid,init,goal,cost,heuristic)

[[0, -1, -1, -1, -1, -1],
 [1, -1, -1, -1, -1, -1],
 [2, -1, -1, -1, -1, -1],
 [3, -1, 8, 9, 10, 11],
 [4, 5, 6, 7, -1, 12]]

# 动态规划 Dynamic Programming

动态规划算法可以以任意点为起点，得到最佳路径。其目标是计算在每一个位置上的最佳的运动方向。得到的这种策略与起点的位置往往时无关的  
动态规划算法的关键在于构建价值函数 value function。其含义是每个位置到达终点位置的最短距离。

In [29]:
def dp_search(grid,init,goal,cost): 

    #定义value函数（矩阵）用于储存各个点到终点的最小步数        
    value = [[99 for col in range(len(grid[0]))] for row in range(len(grid))]

    #定义policy测率函数，用于储存再各个点上，为了到达终点的最佳移动方向
    policy = [["" for col in range(len(grid[0]))] for row in range(len(grid))]    

    #定义update变量并初始化为True，用于标记value函数是否还需要更新    
    update = True
    while update:
        #循环开始时，先将update定义为False，后面如果没有动作，则不再更新
        update = False

        #依次取地图上的每一个点作为探索的中心点        
        for x in range(len(grid)):
            for y in range(len(grid[0])):

                #判断是否为终点，如为终点，则将其value值设为0，并把policy值设为*
                if goal[0] == x and goal[1] == y:
                    if value[x][y] > 0:
                        value[x][y] = 0
                        policy[x][y] = "*"
                        update = True

                #如果不是终点，则判断是否为可达的点
                elif grid[x][y] == 0:
                    #如果时可达的点，则在该中心点上往4个方向，依次移动探索
                    for a in range(len(delta)):
                        x2 = x + delta[a][0]
                        y2 = y + delta[a][1]
                        
                        #如果周围的的点，在地图范围内，且是可达的
                        if x2 >= 0 and x2 <= len(grid)-1 and y2 >=0 and y2 <= len(grid[0])-1 \
                            and grid[x2][y2] == 0:

                            #则更新一下周边点的value值，加上移动的cost
                            v2 = value[x2][y2] + cost

                            #如果周边的点的value值小于还小于中心点的value值
                            #说明目前中心点的value比周边被探索点的value大了2cost以上
                            #应用周边点的value值替换中心点的value值，这样可保证相邻value值得差保持在1个cost
                            if v2 < value[x][y]:
                                update = True                                
                                value[x][y] = v2                                     
                                policy[x][y] = delta_name[a]
                                
    return value, policy

In [30]:
dp_search(grid,init,goal,cost)

([[11, 99, 7, 6, 5, 4],
  [10, 99, 6, 5, 4, 3],
  [9, 99, 5, 4, 3, 2],
  [8, 99, 4, 3, 2, 1],
  [7, 6, 5, 4, 99, 0]],
 [['v', '', 'v', 'v', 'v', 'v'],
  ['v', '', 'v', 'v', 'v', 'v'],
  ['v', '', 'v', 'v', 'v', 'v'],
  ['v', '', '>', '>', '>', 'v'],
  ['>', '>', '^', '^', '', '*']])

# 左转策略

在实际驾驶中，左转相对于其他动作更加复杂，因此需将左转的cost设置的更高。
* 在规划路线时，则需要在2D的value和policy矩阵上增加一个维度，用于储存动作信息
* 增加action变量，表示车辆保持直行，还是左转或者右转，和forwad变量的不同在于，action是再车辆坐标系下

In [32]:
forward = [[-1,  0], # go up
           [ 0, -1], # go left
           [ 1,  0], # go down
           [ 0,  1]] # go right
forward_name = ['up', 'left', 'down', 'right']

# action has 3 values: right turn, no turn, left turn
action = [-1, 0, 1]
action_name = ['R', '#', 'L']

grid = [[1, 1, 1, 0, 0, 0],
        [1, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 1],
        [1, 1, 1, 0, 1, 1]]

init = [4, 3, 0] # given in the form [row,col,direction]
                 # direction = 0: up
                 #             1: left
                 #             2: down
                 #             3: right
                
goal = [2, 0] # given in the form [row,col]

cost = [2, 1, 20] # cost has 3 values, corresponding to making 
                  # a right turn, no turn, and a left turn

In [33]:
def left_turn_policy(grid,init,goal,cost): 
        
    value = [[[999 for col in range(len(grid[0]))] for row in range(len(grid))],
             [[999 for col in range(len(grid[0]))] for row in range(len(grid))],
             [[999 for col in range(len(grid[0]))] for row in range(len(grid))],
             [[999 for col in range(len(grid[0]))] for row in range(len(grid))]]
    
    policy = [[[" " for col in range(len(grid[0]))] for row in range(len(grid))],
              [[" " for col in range(len(grid[0]))] for row in range(len(grid))],
              [[" " for col in range(len(grid[0]))] for row in range(len(grid))],
              [[" " for col in range(len(grid[0]))] for row in range(len(grid))]]
    
    policy2D = [[" " for col in range(len(grid[0]))] for row in range(len(grid))]
    
    change = True
    while change:
        change = False
        
        for x in range(len(grid)):
            for y in range(len(grid[0])):
                for orientation in range(4):

                    if goal[0] == x and goal[1] == y:
                        if value[orientation][x][y] > 0:
                            value[orientation][x][y] = 0
                            policy[orientation][x][y] = "*"
                            change = True

                    elif grid[x][y] == 0:
                        for i in range(3):
                            o2 = (orientation + action[i]) % 4
                            x2 = x + forward[o2][0]
                            y2 = y + forward[o2][1]

                            if x2 >= 0 and x2 <= len(grid)-1 and y2 >=0 and y2 <= len(grid[0])-1 \
                                and grid[x2][y2] == 0:
                                v2 = value[o2][x2][y2] + cost[i]

                                if v2 < value[orientation][x][y]:
                                    change = True
                                    value[orientation][x][y] = v2                               
                                    policy[orientation][x][y] = action_name[i]

    x = init[0]
    y = init[1]
    orientation = init[2]
    
    policy2D[x][y]=policy[orientation][x][y]

    while policy[orientation][x][y] != "*":
        if policy[orientation] == "#":
            o2 = orientation
        elif policy[orientation][x][y] == "R":
            o2 = (orientation -1) % 4
        elif policy[orientation][x][y] == "L":
            o2 = (orientation +1) % 4
        x = x + forward[o2][0]
        y = y + forward[o2][1]
        orientation = o2
        policy2D[x][y] = policy[orientation][x][y]
                                    
    return policy2D

In [34]:
left_turn_policy(grid,init,goal,cost)

[[' ', ' ', ' ', 'R', '#', 'R'],
 [' ', ' ', ' ', '#', ' ', '#'],
 ['*', '#', '#', '#', '#', 'R'],
 [' ', ' ', ' ', '#', ' ', ' '],
 [' ', ' ', ' ', '#', ' ', ' ']]